# 2. Árboles de Decisión <a id="6"></a>

## Leer el Conjunto de Datos

Cargar los datos y guardarlos en el dataframe `df3`:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import train_test_split
%matplotlib inline

In [ ]:
# ruta de datos y leer los datos

path3='datos/drug200.csv'
df3 = pd.read_csv(path3)
df3.head()

Tamaño y forma del conjunto de datos.

In [ ]:
print('Tamaño: ', df3.size)
print('Forma: ', df3.shape)

## Pre-procesamiento de los Datos

Usar el dataframe `df3`, leído con *Pandas*, con los datos del conjunto de datos `drug200.csv` para convertirlo y declarar las siguientes variables:

* **X3** como la Matriz de características.
* **y3** como el Vector de respuesta u objetivo.

Para la matriz de características se va a incorporar todas las columnas excepto la última que corresponde al vector de respuesta.

In [ ]:
X3 = df3[['Age', 'Sex', 'BP', 'Cholesterol', 'Na_to_K']].values
X3[0:5]

Se debe obserbar que algunas características en este conjunto de datos son categóricas, como `Sex` o `BP`. Desafortunadamente, el módulo de *Árboles de Decisión* de la biblioteca *Scikit-learn* no maneja variables categóricas. Sin embrago, es posible convertir estas características en valores numéricos utilizando el método de *Pandas* `pandas.get_dummies()`.

De esta forma se procederá a convertir las variables categóricas en variables dummy/indicadoras.

In [ ]:
codificador_etiqueta_Sex = preprocessing.LabelEncoder()
codificador_etiqueta_Sex.fit(['F','M'])
X3[:,1] = codificador_etiqueta_Sex.transform(X3[:,1]) 


codificador_etiqueta_BP = preprocessing.LabelEncoder()
codificador_etiqueta_BP.fit([ 'LOW', 'NORMAL', 'HIGH'])
X3[:,2] = codificador_etiqueta_BP.transform(X3[:,2])


codificador_etiqueta_Cholesterol = preprocessing.LabelEncoder()
codificador_etiqueta_Cholesterol.fit([ 'NORMAL', 'HIGH'])
X3[:,3] = codificador_etiqueta_Cholesterol.transform(X3[:,3]) 

X3[0:5]

Una vez arreglada la matriz de características **X** su puede proceder a crear el vector de destino **y**.

In [ ]:
y3 = df3["Drug"]
y3[0:5]

## Configuración del Modelo

A continuación, se procederá a la configuración del Árbol de Decisiones. Para ello, se usará la división de los conjuntos de entrenamiento y de prueba para árbol de decisión. 

Para este caso en particular, se declararán los 4 parámetros de salida con los nombres: `X3_entrena`, `X3_prueba`, `y3_entrena` e `y3_prueba`.

A la función `train_test_split` se le ingresarán los siguientes parámetros: `X3`, `y3`, `test_size=0.3` y `random_state=3`.

Recordar que `X3` e `y3` son los arreglos requeridos para poder realizar la divsión, `test_size` representa la proporción del conjunto de datos de prueba y `random_state` asegura que obtengamos las mismas divisiones.


In [ ]:
X3_entrena, X3_prueba, y3_entrena, y3_prueba = train_test_split(X3, y3, test_size=0.3, random_state=3)
print ('Conjunto de Entrenamiento set:', X3_entrena.shape,  y3_entrena.shape)
print ('Conjunto de Prueba:', X3_prueba.shape,  y3_prueba.shape)

## Modelado

Primero que nada, hay que importar y cargar el módulo `tree` que habilita el clasificador que implementa el modelo del Árbol de Decisiones.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

A continuación, se procede a crear una instancia de `DecisionTreeClassifier` llamada `arbol_modelo`. Dentro del clasificador, se va a configurar el parámetro `criterion="entropy"` para que se pueda observar ver la ganancia de información de cada nodo.

In [ ]:
arbol_modelo = DecisionTreeClassifier(criterion="entropy", max_depth = 6)
arbol_modelo

A continuación, se procederá a ajustar el modelo con los conjuntos de datos de entrenamiento `X3_entrena` e `y3_entrena`.

In [ ]:
arbol_modelo.fit(X3_entrena,y3_entrena)

## Pronóstico

Una vez entrenado el modelo se puede proceder a realizar algunas predicciones con el conjunto de datos de prueba, las cuales serán almacenadas en una variable llamada `arbol_pronostico`.

In [ ]:
arbol_pronostico = arbol_modelo.predict(X3_prueba)

Ahora se puede imprimir `arbol_pronostico` e `y3_prueba` para comparar visualmente la predicción con los valores reales.

In [ ]:
print (arbol_pronostico [0:5])
print (y3_prueba [0:5])

## Evaluación

A continuación, el siguiente paso consiste en utilizar las métricas de **sklearn** y verifiquemos la precisión del modelo.

In [ ]:
print("Precición del modelo basado en árbol de decisiones: ", metrics.accuracy_score(y3_prueba, arbol_pronostico))

La **puntuación de la precisión del clasificador** calcula la precisión del subconjunto conformado por el conjunto de etiquetas pronosticado para una muestra dada, el cual debería coincidir exactamente con el conjunto de etiquetas correspondiente en **y_prueba** o **y_verdadrero** o **y_real**.

En la clasificación multi-etiqueta, la función devuelve la precisión del subconjunto. Si todo el conjunto de etiquetas pronosticadas para una muestra coincide estrictamente con el verdadero conjunto de etiquetas, entonces la precisión del subconjunto es $1,0$; de lo contrario es $0,0$.

Una forma de calcular la precisión del modelo sin usar **sklearn** sería de la siguiente forma.

In [ ]:
precision_alternativo = np.sum(np.equal(y3_prueba, arbol_pronostico)) / len(y3_prueba)
precision_alternativo